In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
berk.csv
berk-test.csv
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
samples
state_transition_matrix.npy
test_predictions.csv
title.csv


In [3]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_PA-100K/annotation/annotation.mat')
attributes_list = []
for i in range(data['attributes'].shape[0]):
    attributes_list.append(data['attributes'][i][0][0])
print(attributes_list)

['Female', 'AgeOver60', 'Age18-60', 'AgeLess18', 'Front', 'Side', 'Back', 'Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack', 'HoldObjectsInFront', 'ShortSleeve', 'LongSleeve', 'UpperStride', 'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LowerStripe', 'LowerPattern', 'LongCoat', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots']


In [4]:
low_level = [15,16,17,18,19,20]
mid_level = [7,8,9,10,11,12,13,14,21,22,23,24,25]
high_level = [0,1,2,3,4,5,6]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)

The total attributes labeled:  26
['UpperStride', 'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LowerStripe', 'LowerPattern', 'Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack', 'HoldObjectsInFront', 'ShortSleeve', 'LongSleeve', 'LongCoat', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots', 'Female', 'AgeOver60', 'Age18-60', 'AgeLess18', 'Front', 'Side', 'Back']


In [5]:
filename = r"../results/PA-100K_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 26))
for i in range(length):
    data_y[i] = np.array(data[i, 1:27], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
y_test = data_y[90000:]
y_test_hiar = data_y_hiar[90000:]
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the y_test is:  (10000, 26)
The shape of the y_test is:  (10000, 26)


# Defining the function

In [6]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0).count(True)
        #print(P,',', N,',', TP,',', TN)
        if P != 0:
            res += TP/P + TN/N
        else:
            res += TN/N
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            M_ += 1
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

# Metrics

In [10]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary26_binary26_final50iter_model_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test_hiar
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))


###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary26_binary26_trainable__final50iter_model_predictions_imagenet_test_PA-100K.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")
label = y_test
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.647131606568794
0.6260379964480025
0.7895253174603469
0.7246015079365342
0.755671488002179
GoogLeNet
0.6567051470450717
0.6357400238650315
0.7985230158730449
0.7292011904762148
0.7622893338696806
